In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")
BASE_URL = os.getenv("BASE_URL")

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="deepseek-v3",
    model_provider="openai",
    api_key=DASHSCOPE_API_KEY,
    base_url=BASE_URL,
)

In [3]:
question = "你好，请你介绍一下你自己"

result = model.invoke(question)
print(result.content)

你好呀！我是 **DeepSeek Chat**，由 **深度求索（DeepSeek）** 研发的智能 AI 助手。我可以帮你解答各种问题，包括学习、工作、编程、生活百科等，还能处理 **128K 上下文** 的长文本，支持 **文件上传**（如 PDF、Word、Excel 等）并从中提取信息进行分析。  

### ✨ **我的特点**：
- **免费使用**：目前没有任何收费计划，放心使用！  
- **知识丰富**：我的知识截止到**2024年7月**，可以为你提供较新的信息。  
- **超长上下文**：能记住 **128K**（约30万字）的对话内容，适合处理长文档或复杂问题。  
- **文件阅读**：支持上传 **PDF、Word、Excel、PPT、txt** 等文件，帮你总结、解析或提取关键信息。  
- **编程助手**：能帮你写代码、调试、优化算法，支持多种编程语言。  
- **生活助手**：无论是旅行攻略、美食推荐、健康建议，还是情感交流，我都能陪你聊聊！  

### 🚀 **你可以这样使用我**：
- **学习**：解答数学题、写论文大纲、翻译外语等。  
- **工作**：写商业计划、优化简历、生成 PPT 大纲等。  
- **编程**：写代码、调试报错、优化性能等。  
- **日常**：查天气、推荐电影、陪聊解闷等。  

有什么想问的？尽管开口，我会尽力帮你！😊


## 预构建 API 快速构建智能体

In [3]:
import requests
import json
from pydantic import BaseModel, Field
from langchain_core.tools import tool


class WeatherQuery(BaseModel):
    loc: str = Field(description="The location name of the city")


@tool(args_schema=WeatherQuery)
def get_weather(loc):
    """ 查询即时天气函数
    :param loc: 必要参数，字符串类型，用户查询天气的具体城市名称，\
    注意，中国的城市需要用对应成是的英文名称代替，例如查询北京市额天气，则loc参数需要输入'Beijing'
    :return: OpenWeather API 查询即时天气的结果，具体URL请求地址为: https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析后的JSON格式对象，并用字符串的形式表示，其中包含了全部重要的天气信息
    """
    url = "https://api.openweathermap.org/data/2.5/weather"

    params = {
        "q": loc,
        "appid": os.getenv("OPENWEATHER_API_KEY"),
        "units": "metric",
        "lang": "zh_cn",
    }

    response = requests.get(url, params=params)

    data = response.json()

    return json.dumps(data)

In [4]:
from langgraph.prebuilt import create_react_agent

tools = [get_weather]

agent = create_react_agent(model=model, tools=tools)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "你好，请介绍一下你自己"}]}
)
response

{'messages': [HumanMessage(content='你好，请介绍一下你自己', additional_kwargs={}, response_metadata={}, id='3ce22ebc-0cb6-4cb4-a95d-df1a2e90cdcf'),
  AIMessage(content='你好！我是一个智能助手，专门用来帮助解答各种问题、提供信息或执行任务。我可以查询天气、提供建议、解答常识问题，甚至帮助你进行语言翻译等。如果你有任何问题或需要帮助的地方，随时告诉我！ \n\n例如：\n- 想查天气？告诉我城市名称就行。\n- 需要学习或工作建议？我也可以帮忙。\n- 或者单纯想聊聊天？我随时在线！ \n\n有什么可以帮你的吗？ 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 256, 'total_tokens': 351, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-4ab89678-1d61-99f4-9dae-d13ab46bc24f', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--4968a7d3-7334-4881-ab57-c297d53c96e5-0', usage_metadata={'input_tokens': 256, 'output_tokens': 95, 'total_tokens': 351, 'input_token_details': {}, 'output_token_details': {}})]}

In [6]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "请问北京今天天气如何？"}]}
)
response

{'messages': [HumanMessage(content='请问北京今天天气如何？', additional_kwargs={}, response_metadata={}, id='75612f2c-349b-4c22-afa0-3790994f33a7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_d10390745b224195aaf53d', 'function': {'arguments': '{"loc":"Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 257, 'total_tokens': 276, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-c2a42d83-85b3-91e0-9328-17d4980a9896', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--78cf78ac-342f-45f2-bfb1-ad6534824dd7-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_d10390745b224195aaf53d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 19, 'total_tokens': 276, 'input_token_details': {}, 'output_t

In [5]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "请问北京和杭州哪里更热？"}]}
)
response

{'messages': [HumanMessage(content='请问北京和杭州哪里更热？', additional_kwargs={}, response_metadata={}, id='6749edef-a356-4a79-a45f-ba4bf4c67215'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8416af1a83f44436863a88', 'function': {'arguments': '{"loc": "Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_d40fe763cef040d8921dc3', 'function': {'arguments': '{"loc": "Hangzhou"}', 'name': 'get_weather'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 259, 'total_tokens': 300, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-ba746fb7-9686-9ef2-965f-b6113d61974a', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4fa1ecbc-8ac8-464b-ae8e-60deb6b56c45-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_8416af1a83f444368

In [7]:
class WriteQuery(BaseModel):
    content: str = Field(description="需要写入文档的具体内容")


@tool(args_schema=WriteQuery)
def write_file(content: str) -> str:
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return: 是否写入成功
    """

    return "已成功写入本地文件"

In [8]:
tools = [get_weather, write_file]

agent = create_react_agent(model=model, tools=tools)

In [9]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "human",
                "content": "你好请帮我查询北京和杭州的天气，并将其写入本地文件中",
            }
        ]
    }
)
response

{'messages': [HumanMessage(content='你好请帮我查询北京和杭州的天气，并将其写入本地文件中', additional_kwargs={}, response_metadata={}, id='b62fa1d7-3112-44b7-8e6f-08589031c726'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_62415aff5a09465d9c1d7e', 'function': {'arguments': '{"loc": "Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}, {'id': 'call_0a79283e7ede427bb20156', 'function': {'arguments': '{"loc": "Hangzhou"}', 'name': 'get_weather'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 363, 'total_tokens': 404, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-3e5f1127-c0b8-9c7d-a2a5-ff43a184d9c8', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1dee103f-3053-4e43-92ae-57fe7ada9433-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_624

In [10]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(max_results=5, topic="general")

tools = [search_tool]

search_agent = create_react_agent(model=model, tools=tools)

In [11]:
response = search_agent.invoke(
    {
        "messages": [
            {"role": "human", "content": "帮我搜索最近 OpenAI CEO在访谈中的核心观点。"}
        ]
    }
)
response

{'messages': [HumanMessage(content='帮我搜索最近 OpenAI CEO在访谈中的核心观点。', additional_kwargs={}, response_metadata={}, id='52e0de7f-f36d-4fed-af48-1c2065b6e592'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_a4d14fc4028046abaa8444', 'function': {'arguments': '{"query":"OpenAI CEO 访谈 核心观点","search_depth":"advanced","time_range":"week","topic":"news"}', 'name': 'tavily_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 1189, 'total_tokens': 1230, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-90f74ab5-87c7-9bff-8e0c-d70057e1b190', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--17cec9ef-5da4-47e9-85fb-c8bb28016834-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'OpenAI CEO 访谈 核心观点', 'search_depth': 'advanced', 'time_range': 'week', 'topic': 'news'},

In [13]:
print(response["messages"][-1].content)

最近OpenAI CEO Sam Altman在访谈中分享了一些核心观点，主要包括：

1. **技术与团队的重要性**：Altman强调了在早期阶段，年轻、有冲劲的团队成员比经验丰富的老将更为重要。他更看重个人的成长潜力和实际成就，而非传统履历。

2. **未来十年的技术方向**：他认为未来十年与过去最大的区别在于个人或小团队的产出数量和质量将达到全新高度，强调了技术创新和团队协作的重要性。

3. **OpenAI的挑战**：Altman坦言担任OpenAI CEO的挑战在于需要同时处理多个重大决策，并且面临来自大公司的竞争压力。

4. **对AGI的信念**：他谈到了推动AGI（通用人工智能）发展的信念和团队的核心价值观，认为这是未来最值得投入的技术方向。

更多详细内容可以参考[新浪财经的报道](https://finance.sina.cn/stock/jdts/2025-07-13/detail-inffhtrt7335082.d.html?vt=4&cid=76993&node_id=76993)。


In [15]:
# 限制工具调用次数

from langgraph.errors import GraphRecursionError


try:
    response = agent.invoke(
        {"messages": [{"role": "user", "content": "请问北京今天天气如何？"}]},
        {"recursion_limit": 4},
    )
except GraphRecursionError:
    print("Agent stopped due to max iterations.")

response

{'messages': [HumanMessage(content='请问北京今天天气如何？', additional_kwargs={}, response_metadata={}, id='9a21f90a-9798-4e44-b09f-e212733ecab6'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_37a8eb4dda3944b8b55632', 'function': {'arguments': '{"loc":"Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 355, 'total_tokens': 374, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-714a212c-8b3e-9470-90da-a5c3db6f068a', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--314eb062-ea37-44d7-8374-013718dda3ec-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_37a8eb4dda3944b8b55632', 'type': 'tool_call'}], usage_metadata={'input_tokens': 355, 'output_tokens': 19, 'total_tokens': 374, 'input_token_details': {}, 'output_t

## 记忆

In [5]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent

checkpointer = InMemorySaver()

tools = [get_weather]

agent = create_react_agent(model=model, tools=tools, checkpointer=checkpointer)

In [6]:
config = {
    "configurable": {
        "thread_id": 1,
    }
}

In [9]:
response = agent.invoke(
    {"messages": [{"role": "human", "content": "你好，我叫陈明好久不见"}]},
    config,
)
response

{'messages': [HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='f225c98f-0b01-4bdd-8785-88f74d7d4906'),
  HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='b59d8937-11dd-4892-9f06-1c94a8db7ebf'),
  AIMessage(content='你好，陈明！好久不见，最近过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 267, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-544ce87c-53e6-9390-878c-a45de145fc13', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--708faea8-546c-42b8-a2f9-b8708f49026b-0', usage_metadata={'input_tokens': 267, 'output_tokens': 12, 'total_tokens': 279, 'input_token_details': {}, 'output_token_details': {}})]}

In [10]:
latest = agent.get_state(config)
latest

StateSnapshot(values={'messages': [HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='f225c98f-0b01-4bdd-8785-88f74d7d4906'), HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='b59d8937-11dd-4892-9f06-1c94a8db7ebf'), AIMessage(content='你好，陈明！好久不见，最近过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 267, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-544ce87c-53e6-9390-878c-a45de145fc13', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--708faea8-546c-42b8-a2f9-b8708f49026b-0', usage_metadata={'input_tokens': 267, 'output_tokens': 12, 'total_tokens': 279, 'input_token_details': {}, 'output_token_details': {}})]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f062221-79a

In [11]:
response = agent.invoke(
    {"messages": [{"role": "human", "content": "你好，你还记得我的名字吗？"}]},
    config,
)
response

{'messages': [HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='f225c98f-0b01-4bdd-8785-88f74d7d4906'),
  HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='b59d8937-11dd-4892-9f06-1c94a8db7ebf'),
  AIMessage(content='你好，陈明！好久不见，最近过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 267, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-544ce87c-53e6-9390-878c-a45de145fc13', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--708faea8-546c-42b8-a2f9-b8708f49026b-0', usage_metadata={'input_tokens': 267, 'output_tokens': 12, 'total_tokens': 279, 'input_token_details': {}, 'output_token_details': {}}),
  HumanMessage(content='你好，你还记得我的名字吗？', additional_kwargs={}, response_metadata={}, id='84095d25-0803-4eb5-b828-e724165ffb

In [13]:
latest = agent.get_state(config)
latest

StateSnapshot(values={'messages': [HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='f225c98f-0b01-4bdd-8785-88f74d7d4906'), HumanMessage(content='你好，我叫陈明好久不见', additional_kwargs={}, response_metadata={}, id='b59d8937-11dd-4892-9f06-1c94a8db7ebf'), AIMessage(content='你好，陈明！好久不见，最近过得怎么样？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 267, 'total_tokens': 279, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-544ce87c-53e6-9390-878c-a45de145fc13', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--708faea8-546c-42b8-a2f9-b8708f49026b-0', usage_metadata={'input_tokens': 267, 'output_tokens': 12, 'total_tokens': 279, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='你好，你还记得我的名字吗？', additional_kwargs={}, response_metadata={}, id='84095d25-0803-4eb5-